<a href="https://colab.research.google.com/github/sergeyman/py_nlp_nltk_1/blob/main/PMLB_SE_Ch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Pocket
# My Applications
# In Development
# NAME		PLATFORM		CONSUMER KEY
# Newsfeed	Android - Mobile	96779-785f8e21498d91359ddbeb55
# Newsfeed	Android - Tablet	96779-3eb65576097636bbae7de71d
# Newsfeed	Extension		96779-ceabd8029fdf6078580f4ccc
# Newsfeed	Windows - Desktop	96779-4bdbc0dcdfe1e3e89da9b55f
# Newsfeed	Web			96779-f71d606f7d5a34fa3a1cb015
# Newsfeed	Mobile (other)		96779-a30d61857b2c0f2c0b6cc914
# Newsfeed	Desktop (other)		96779-fcaf090432e3b6a8a08e644c


import requests
import pandas as pd
import json
pd.set_option('display.max_colwidth', 200)

## Get Stories from Pocket

In [2]:
CONSUMER_KEY = '96779-4bdbc0dcdfe1e3e89da9b55f'  #'your_pocket_consumer_key'

In [3]:
auth_params = {'consumer_key': CONSUMER_KEY, 'redirect_uri': 'https://twitter.com/acombs'}

In [4]:
tkn = requests.post('https://getpocket.com/v3/oauth/request', data=auth_params)

In [5]:
tkn.text

'code=223b8284-5921-fdc1-108b-967cae'

In [6]:
ACCESS_CODE = tkn.text.split('=')[1]

In [7]:
ACCESS_CODE

'223b8284-5921-fdc1-108b-967cae'

In [8]:
URL = 'https://getpocket.com/auth/authorize?request_token=' + ACCESS_CODE + '&redirect_uri=https%3A//www.twitter.com/acombs'

In [9]:
URL

'https://getpocket.com/auth/authorize?request_token=223b8284-5921-fdc1-108b-967cae&redirect_uri=https%3A//www.twitter.com/acombs'

In [10]:
usr_params = {'consumer_key': CONSUMER_KEY, 'code': ACCESS_CODE}

In [11]:
usr = requests.post('https://getpocket.com/v3/oauth/authorize', data=usr_params)

In [12]:
usr.text

'403 Forbidden'

In [ ]:
ACCESS_TOKEN = usr.text.split('=')[1].split('&')[0]

In [ ]:
ACCESS_TOKEN

### Get 'no' stories

In [ ]:
no_params = {'consumer_key': CONSUMER_KEY,
'access_token': ACCESS_TOKEN,
'tag': 'n'}

In [ ]:
no_result = requests.post('https://getpocket.com/v3/get', data=no_params)

In [ ]:
no_result.text

In [ ]:
no_jf = json.loads(no_result.text)

In [ ]:
no_jd = no_jf['list']

In [ ]:
no_jd

In [ ]:
no_urls=[]
for i in no_jd.values():
    no_urls.append(i.get('resolved_url'))

In [ ]:
no_urls

In [ ]:
len(no_urls)

In [ ]:
no_uf = pd.DataFrame(no_urls, columns=['urls'])

In [ ]:
no_uf

In [ ]:
no_uf = no_uf.assign(wanted = lambda x: 'n')

In [ ]:
no_uf

### Get 'yes' stories

In [ ]:
yes_params = {'consumer_key': CONSUMER_KEY,
'access_token': ACCESS_TOKEN,
'tag': 'y',
'state': 'archive'}

In [ ]:
yes_result = requests.post('https://getpocket.com/v3/get', data=yes_params)

In [ ]:
yes_result.text

In [ ]:
yes_jf = json.loads(yes_result.text)

In [ ]:
yes_jd = yes_jf['list']

In [ ]:
yes_jf

In [ ]:
yes_urls=[]
for i in yes_jd.values():
    yes_urls.append(i.get('resolved_url'))

In [ ]:
len(yes_urls)

In [ ]:
yes_urls

In [ ]:
yes_uf = pd.DataFrame(yes_urls, columns=['urls'])

In [ ]:
yes_uf

In [ ]:
yes_uf = yes_uf.assign(wanted = lambda x: 'y')

In [ ]:
yes_uf

In [ ]:
df = pd.concat([yes_uf, no_uf])

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

## Download Articles to Run Through Model

In [ ]:
import urllib

In [ ]:
EMBEDLY_KEY = 'you_embedly_api_key_here'

def get_html(x):
    try:
        qurl = urllib.parse.quote(x)
        rhtml = requests.get('https://api.embedly.com/1/extract?url=' + qurl + '&key=' + EMBEDLY_KEY)
        ctnt = json.loads(rhtml.text).get('content')
    except:
        return None
    return ctnt

In [ ]:
df.loc[:,'html'] = df['urls'].map(get_html)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

### Extract the text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def get_text(x):
    soup = BeautifulSoup(x, 'html5lib')
    text = soup.get_text()
    return text

In [ ]:
df.loc[:,'text'] = df['html'].map(get_text)

In [ ]:
df

# Implement Tfid Vectorization & Fit Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,3), stop_words='english', min_df=3)

In [ ]:
tv = vect.fit_transform(df['text'])

In [ ]:
clf = LinearSVC()

In [ ]:
model = clf.fit(tv, df['wanted'])

## Pull New Articles from Google Drive Sheet to Evaluate

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
JSON_API_KEY = r'/your/path/to/api/key/file.json'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_API_KEY, scope)
gc = gspread.authorize(credentials)

In [ ]:
# must share with client_email in json api key file
ws = gc.open("NewStories")

In [ ]:
sh = ws.sheet1

In [ ]:
zd = list(zip(sh.col_values(2),sh.col_values(3), sh.col_values(4)))

In [ ]:
zd

In [ ]:
zf = pd.DataFrame(zd, columns=['title','urls','html'])

In [ ]:
zf.replace('', pd.np.nan, inplace=True)
zf.dropna(inplace=True)

In [ ]:
zf

In [ ]:
zf.loc[:,'text'] = zf['html'].map(get_text)

In [ ]:
zf.reset_index(drop=True, inplace=True)

In [ ]:
test_matrix = vect.transform(zf['text'])

In [ ]:
test_matrix

In [ ]:
results = pd.DataFrame(model.predict(test_matrix), columns=['wanted'])

In [ ]:
results

In [ ]:
rez = pd.merge(results,zf, left_index=True, right_index=True)

In [ ]:
rez

In [ ]:
for i, w, t in zip(rez[rez['wanted']=='y'].index, rez[rez['wanted']=='y']['wanted'], rez[rez['wanted']=='y']['title']):
    print(i, w, t)

In [ ]:
# those items to change to no
change_to_no = [130, 145, 148, 163, 178, 199, 219, 222, 223, 226, 235, 279, 348, 357, 427, 440, 542, 544, 546, 568, 614, 619, 660, 668, 679, 686, 740, 829]

In [ ]:
for i, w, t in zip(rez[rez['wanted']=='n'].index, rez[rez['wanted']=='n']['wanted'], rez[rez['wanted']=='n']['title']):
    print(i, w, t)

In [ ]:
# those items to change to yes
change_to_yes = [0, 9, 29, 35, 42, 71, 110, 190, 319, 335, 344, 371, 385, 399, 408, 409, 422, 472, 520, 534, 672]

In [ ]:
rez

In [ ]:
for i in rez.iloc[change_to_yes].index:
    rez.iloc[i]['wanted'] = 'y'

In [ ]:
for i in rez.iloc[change_to_no].index:
    rez.iloc[i]['wanted'] = 'n'

In [ ]:
rez

In [ ]:
df

In [ ]:
combined = pd.concat([df[['wanted', 'text']], rez[['wanted', 'text']]])

In [ ]:
combined

In [ ]:
tvcomb = vect.fit_transform(combined['text'], combined['wanted'])

In [ ]:
model = clf.fit(tvcomb, combined['wanted'])

In [ ]:
model

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open(r'/path/to/output/your/pickle/model_file.p', 'wb'))

In [ ]:
pickle.dump(vect, open(r'/path/to/your/pickle/vect_file.p', 'wb'))